Using RAGAS metrics to help evaluate our RAG LLM.
Below are the links that will help you to start usinG RAGAS
- link: https://github.com/rajshah4/LLM-Evaluation/blob/main/ragas_quickstart.ipynb
- link: https://docs.ragas.io/en/stable/getstarted/evaluation.html

# Testing RAGAS

In [3]:
from datasets import Dataset 
import os
from dotenv import load_dotenv  # Import the function to load .env variables
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision, context_utilization, context_entity_recall
import openai
import chromadb
from chromadb.utils import embedding_functions

# Load environment variables from the .env file
load_dotenv()

# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")  # Load API key from environment
openai_client = openai  # Using the openai module directly

# Initialize ChromaDB client and load collection
def load_collection():
    CHROMA_DATA_PATH = "eskwe"
    COLLECTION_NAME = "eskwe_embeddings"
    client_chromadb = chromadb.PersistentClient(path=CHROMA_DATA_PATH)
    
    # Load API key from environment for embedding function
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-ada-002")
    
    collection = client_chromadb.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=openai_ef,
        metadata={"hnsw:space": "cosine"}
    )
    return collection

collection = load_collection()

# Function to return the best matching data in the collection based on user input
def return_best_data(user_input, collection, n_results=1):
    query_result = collection.query(query_texts=[user_input], n_results=n_results)
    if not query_result['ids'] or not query_result['ids'][0]:
        return []
    
    # Collect the top N results
    results = []
    for i in range(n_results):
        if i < len(query_result['ids'][0]):
            top_result_document = query_result['documents'][0][i]
            results.append(top_result_document)
    return results

# Function to generate a conversational response using OpenAI API with document-based initial response
def generate_conversational_response(user_input, collection):
    related_articles = return_best_data(user_input, collection, n_results=1)
    
    if not related_articles:
        return "I couldn't find any relevant articles based on your input."
    
    # Use the retrieved document to form the initial response
    document_content = related_articles[0][:2000]  # Limit the document content to a reasonable length

    # Generate a conversational response using the document content
    conversation_prompt = (
        f"You are an expert in Data Science. Based on the following information, please provide a friendly and conversational explanation. No need to mention the article. Provide code as much as possible.:\n\n"
        f"{document_content}"
    )

    try:
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in Data Science and a friendly assistant who provides clear and engaging explanations. No need to mention the article."},
                {"role": "user", "content": conversation_prompt}
            ],
            max_tokens=1000,
        )
        final_response = response.choices[0].message.content
        return final_response
    
    except Exception as e:
        return f"An error occurred with OpenAI API: {e}"

# Main Execution
user_input = "How does Retrieval-Augmented Generation (RAG) work in Large Language Models (LLMs)?"  # You can change this input manually
print(f"User Input: {user_input}")

# Generate the AI response
ai_answer = generate_conversational_response(user_input, collection)
print(f"AI Answer: {ai_answer}")

# Retrieve contexts from ChromaDB
contexts = return_best_data(user_input, collection)
print(f"Contexts: {contexts}")

# Create the data_samples dictionary
data_samples = {
    'question': [user_input],
    'answer': [ai_answer],
    'contexts': [contexts]
}

# Print the final data_samples dictionary
print("Data Samples:", data_samples)

# Convert to Dataset and evaluate with RAGAS
dataset = Dataset.from_dict(data_samples)
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_utilization

score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_utilization])
score.to_pandas()


User Input: How does Retrieval-Augmented Generation (RAG) work in Large Language Models (LLMs)?


Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

AI Answer: Retrieval Augmented Generation (RAG) is a fascinating technique in the field of Data Science that leverages the strengths of pre-trained large language models (LLMs) like GPT-3 or GPT-4 along with external data sources. Essentially, RAG combines the generative power of these LLMs with specialized data search mechanisms to create a sophisticated system capable of delivering nuanced responses.

Let's break it down into a practical example. Imagine you're an executive at an electronics company and you want to develop a customer support chatbot powered by LLMs. Traditionally, large language models might struggle to provide accurate answers to specific queries related to your company's products or troubleshooting processes because they lack access to organization-specific data and their training data may be outdated.

This is where RAG comes to the rescue! By integrating external data sources into the mix, RAG enables your chatbot to retrieve relevant information in real-time, en

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

,question,answer,contexts,faithfulness,answer_relevancy,context_utilization
0,How does Retrieval-Augmented Generation (RAG) ...,Retrieval Augmented Generation (RAG) is a fasc...,[Title: What is Retrieval Augmented Generation...,0.95,0.95489,1.0


# Using different questions

In [7]:
import pandas as pd
from datasets import Dataset
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_utilization
import openai
import chromadb
from chromadb.utils import embedding_functions

# Load environment variables from the .env file
load_dotenv()

# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")  # Load API key from environment
openai_client = openai  # Using the openai module directly

# Initialize ChromaDB client and load collection
def load_collection():
    CHROMA_DATA_PATH = "eskwe"
    COLLECTION_NAME = "eskwe_embeddings"
    client_chromadb = chromadb.PersistentClient(path=CHROMA_DATA_PATH)
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-ada-002")
    collection = client_chromadb.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=openai_ef,
        metadata={"hnsw:space": "cosine"}
    )
    return collection

collection = load_collection()

# Function to return the best matching data in the collection based on user input
def return_best_data(user_input, collection, n_results=1):
    query_result = collection.query(query_texts=[user_input], n_results=n_results)
    if not query_result['ids'] or not query_result['ids'][0]:
        return []
    
    # Collect the top N results
    results = []
    for i in range(n_results):
        if i < len(query_result['ids'][0]):
            top_result_document = query_result['documents'][0][i]
            results.append(top_result_document)
    return results

# Function to generate a conversational response using OpenAI API with document-based initial response
def generate_conversational_response(user_input, collection):
    related_articles = return_best_data(user_input, collection, n_results=1)
    
    if not related_articles:
        return "I couldn't find any relevant articles based on your input."
    
    # Use the retrieved document to form the initial response
    document_content = related_articles[0][:2000]  # Limit the document content to a reasonable length

    # Generate a conversational response using the document content
    conversation_prompt = (
        f"You are an expert in Data Science. Based on the following information, please provide a friendly and conversational explanation. No need to mention the article. Provide code as much as possible.:\n\n"
        f"{document_content}"
    )

    try:
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in Data Science and a friendly assistant who provides clear and engaging explanations. No need to mention the article."},
                {"role": "user", "content": conversation_prompt}
            ],
            max_tokens=1000,
        )
        final_response = response.choices[0].message.content
        return final_response
    
    except Exception as e:
        return f"An error occurred with OpenAI API: {e}"

# Initialize an empty DataFrame to store scores
scores_df = pd.DataFrame()

# List of sample questions
questions = [
    "What are the key topics covered in Sprint 1?",
    "What are the main topics discussed in Sprint 2?",
    "What should I know about the topics in Sprint 3?",
    "What are the key focus areas in Sprint 4?",
    "How do I install Anaconda to run Python for Data Science?",
    "What is the step-by-step process for installing Anaconda on Windows?",
    "How do I install Anaconda on Mac OS X?",
    "How can I manage environments using Anaconda?",
    "What is the introduction to Credit Card Fraud and Outlier Detection?",
    "How can machine learning be applied to Credit Card Fraud Detection?",
    "How do you use Python, particularly Pandas, for Credit Card Fraud Detection?",
    "What is a simple machine learning model and how do you create one?",
    "Why is Train Test Split important in machine learning?",
    "What is the official Train Test Split documentation in Scikit Learn?",
    "How do tree-based ensembles work in machine learning?",
    "Can you explain the concept of decision trees?",
    "What is a decision tree and how is it used in machine learning?",
    "What are some advanced model evaluation metrics and techniques beyond accuracy?",
    "What is a Confusion Matrix in Machine Learning and how is it used?",
    "How do you create a Precision-Recall Curve in Python?",
    "What does Recall mean in the context of machine learning?",
    "What is the significance of Precision and Recall in machine learning?",
    "How do you conduct EDA and data preparation for an NLP project step by step?",
    "How can you create everyday apps with Streamlit as a data scientist?",
    "Can you provide a quick overview of Large Language Models (LLMs)?",
    "How do you implement text summarization using the ChatGPT API in Python?",
    "What is the NLTK Sentiment Analysis Tutorial for beginners?",
    "How do you use GPT-4 and OpenAI’s functions for text classification?",
    "How can you extract and classify short-text data with the OpenAI API?",
    "What is Named Entity Recognition and how can it enrich text?",
    "How do OpenAI's text generation models work and how can they be used?",
    "What is prompt chaining and how do you use it?",
    "How can you understand and mitigate bias in Large Language Models (LLMs)?",
    "What are the key metrics, methodologies, and best practices for LLM evaluation?",
    "What is Design Thinking and how does it apply to beginners?",
    "How would you learn to code with ChatGPT if you had to start again?",
    "What are the steps to build a storyboard?",
    "How can you become a master storyteller using ChatGPT prompts?",
    "Can you provide a full demo of Retrieval-Augmented Generation (RAG)?",
    "What is Retrieval-Augmented Generation (RAG) and how does it work?",
    "How can you go from basics to advanced concepts in Retrieval-Augmented Generation (RAG)?",
    "How do you work with JSON data in Python?",
    "What is the process for working with JSON files in Python?",
    "What is JSONL and how is it used?",
    "How do you work with JSONL files?",
    "What is the role of vector databases in machine learning, and how do you use them?",
    "How do you get started with text embeddings using the OpenAI API?",
    "What is text classification in Python and how does it work?",
    "What is Bag of Words (BoW) and how is it used in text processing?",
    "How do you install and set up Pandas according to the official documentation?",
    "Which machine learning role is right for you?",
    "What are 10 clustering algorithms that can be implemented in Python?",
    "What are the key clustering algorithms that all data scientists should know?",
    "What is unsupervised learning and how does it relate to data clustering?",
    "How can you create master visualizations with Matplotlib in Python?",
    "What are the steps for data preprocessing in machine learning?"
]


# Process each question
for user_input in questions:
    print(f"User Input: {user_input}")

    # Generate the AI response
    ai_answer = generate_conversational_response(user_input, collection)
    print(f"AI Answer: {ai_answer}")

    # Retrieve contexts from ChromaDB
    contexts = return_best_data(user_input, collection)
    print(f"Contexts: {contexts}")

    # Create the data_samples dictionary
    data_samples = {
        'question': [user_input],
        'answer': [ai_answer],
        'contexts': [contexts]
    }

    # Convert to Dataset and evaluate with RAGAS
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_utilization])

    # Convert score to DataFrame and append to the scores_df
    score_df = score.to_pandas()
    scores_df = pd.concat([scores_df, score_df], ignore_index=True)

# Print the final DataFrame with all scores
scores_df


User Input: What are the key topics covered in Sprint 1?
AI Answer: Hey there! 🌟 In Sprint 1, we will be diving into some exciting topics related to Data Science and Machine Learning.

First, we will start with the main topic: Introduction to Data Science and Machine Learning. This will lay the foundation for understanding the rest of the sprint's content.

Then, we have a list of subtopics that we'll be covering:
- Python Fundamentals: We will go over the basics of Python programming, an essential skill for any data scientist.
- Pandas: Data Wrangling Techniques: Pandas is a powerful library for data manipulation and analysis in Python.
- Data Distributions: Understanding different types of data distributions is crucial for making informed decisions in data analysis.
- Data Visualizations: Visualizing data is key to gaining insights and telling stories from data effectively.
- Exploratory Data Analysis: We will learn how to explore and understand our data before diving into any modeli

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

User Input: What are the main topics discussed in Sprint 2?
AI Answer: Absolutely! In Sprint 2, the main focus is on Machine Learning Techniques and Model Evaluation. This is a crucial aspect of data science where we aim to build and evaluate models that can make predictions based on patterns within data.

The Subtopics covered in Sprint 2 are:

1. Introduction to Credit Card Fraud and Outlier Detection: Understanding how to detect fraudulent transactions and outliers in datasets, which is essential for many real-world applications.

2. Simple Machine Learning Model: Starting with basic machine learning models to understand the fundamentals of building predictive models.

3. Tree-based ensemble models: Dive into more advanced techniques like ensemble modeling, which combines multiple models to improve predictive performance.

4. Resampling techniques: Techniques to address imbalanced datasets, where one class may be much more prevalent than others.

5. Machine Learning Beyond Accuracy:

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

User Input: What should I know about the topics in Sprint 3?
AI Answer: Hey there! 🌟 Let's dive into the exciting world of Sprint 3 with a focus on Applied Natural Language Processing (NLP) and Large Language Models (LLM). This sprint covers a range of fascinating topics that will enhance your skills and knowledge in the realm of data science.

First up, we'll explore NLP basics, where you'll learn about text preprocessing and exploratory data analysis (EDA) using NLP techniques. This sets the foundation for understanding the power of language processing in data science.

Next, we'll delve into creating data applications with Streamlit, a fantastic tool for building interactive web apps with your data analysis projects. You'll bring your insights to life in a user-friendly way.

LLM Overview introduces you to the world of Large Language Models, providing you with a solid understanding of their capabilities and applications in real-world scenarios.

Text Summarization, Sentiment Analysi

...

User Input: How can you create master visualizations with Matplotlib in Python?
AI Answer: Hey there! 🌟 Let's dive into some cool Matplotlib visualizations together.

**Scatter Plot:**
A scatter plot is a classic way to visualize the relationship between two variables. If you have multiple groups in your data, you can color-code them for better understanding. In Matplotlib, you can easily achieve this using `plt.scatterplot()`. Here's a snippet of code for you:

```python
import matplotlib.pyplot as plt
import seaborn as sns

df = sns.load_dataset('iris')
sns.scatterplot(x='sepal_length', y='sepal_width', hue='species', data=df)
plt.show()
```

**Bubble Plot with Encircling:**
Sometimes, you might need to encircle a group of points to highlight their importance. The `encircle()` function below does just that:

```python
import matplotlib.pyplot as plt
import seaborn as sns

# Define the encircle function here

df = sns.load_dataset('iris')
sns.scatterplot(x='sepal_length', y='sepal_wid

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

User Input: What are the steps for data preprocessing in machine learning?
AI Answer: Data preprocessing is a crucial step in Machine Learning as it directly impacts how well our model can learn from the data. It involves cleaning and transforming raw data into a format that is suitable for training a model. 

One common preprocessing task is handling null values, which are missing data points denoted in Python as NaN. These missing values can disrupt the learning process of our model, so it's important to address them. 

To check for null values in a dataset, we use the `isnull()` method. This returns a boolean matrix where True indicates a NaN value. If we want to see the count of null values in each column, we use `df.isnull().sum()`.

A straightforward way to deal with null values is by removing rows or columns that contain them using the `dropna()` method. This method can take parameters like `axis`, `how`, `thresh`, and `subset` to customize how null values are handled. For insta

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

,question,answer,contexts,faithfulness,answer_relevancy,context_utilization
0,What are the key topics covered in Sprint 1?,"Hey there! 🌟 In Sprint 1, we will be diving in...",[Title: Overview and Topics of Sprint 1\nLink:...,0.880000,0.930031,1.0
1,What are the main topics discussed in Sprint 2?,"Absolutely! In Sprint 2, the main focus is on ...",[Title: Overview and Topics of Sprint 2\nLink:...,0.314286,0.990521,1.0
2,What should I know about the topics in Sprint 3?,Hey there! 🌟 Let's dive into the exciting worl...,[Title: Overview and Topics of Sprint 3\nLink:...,0.391304,0.903156,1.0
3,What are the key focus areas in Sprint 4?,Hey there! 👋 Let's delve into the overview and...,[Title: Overview and Topics of Sprint 4\nLink:...,0.533333,0.951715,1.0
4,How do I install Anaconda to run Python for Da...,Hey there! Python is a super popular language ...,[Title: How to Install Anaconda to Run Python ...,1.000000,0.950156,1.0
5,What is the step-by-step process for installin...,Anaconda is a powerful package manager that is...,[Title: Installing Anaconda on Windows Tutoria...,0.812500,0.926971,1.0
6,How do I install Anaconda on Mac OS X?,Hey there! Installing Anaconda on your Mac can...,[Title: Installing Anaconda on Mac OS X\nLink:...,0.947368,0.966159,1.0
7,How can I manage environments using Anaconda?,Managing environments in data science is cruci...,[Title: Managing environments\nLink: https://c...,0.962963,0.908280,1.0
8,What is the introduction to Credit Card Fraud ...,Absolutely! Let's dive into the world of credi...,[Title: Introduction to Credit Card Fraud and ...,0.851852,0.921751,1.0
9,How can machine learning be applied to Credit ...,Hey there! Detecting fraud in credit card tran...,[Title: How to Use Python for Credit Card Frau...,0.888889,0.901834,1.0


# Download as CSV

In [8]:
csv_file_path = "ragas_scores.csv"
scores_df.to_csv(csv_file_path, index=False)

print(f"Scores saved to {csv_file_path}")

Scores saved to ragas_scores.csv
